# Infer-3-Factor-Graphs : Graphes de Facteurs et Inference Discrete

**Serie** : Programmation Probabiliste avec Infer.NET (3/13)  
**Duree estimee** : 45 minutes  
**Prerequis** : Infer-1-Setup, Infer-2-Gaussian-Mixtures

---

## Objectifs

- Comprendre les graphes de facteurs (factor graphs)
- Modeliser l'inference discrete avec Infer.NET
- Resoudre le probleme classique du "Murder Mystery" (MBML Book, Ch.1)
- Comprendre le paradoxe de Monty Hall
- Maitriser `Variable.If` / `Variable.IfNot` pour le conditionnement

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-2-Gaussian-Mixtures](Infer-2-Gaussian-Mixtures.ipynb) | [Infer-4-Bayesian-Networks](Infer-4-Bayesian-Networks.ipynb) |

---

## 1. Configuration

Nous chargeons les packages Infer.NET necessaires pour ce notebook. L'utilisation du compilateur Roslyn (`CompilerChoice.Roslyn`) est indispensable dans l'environnement .NET Interactive pour permettre la compilation dynamique des modeles probabilistes.

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Introduction aux Graphes de Facteurs

### Definition

Un **graphe de facteurs** (factor graph) est une representation graphique d'une distribution de probabilite jointe qui se factorise en un produit de fonctions.

$$P(X_1, X_2, ..., X_n) = \frac{1}{Z} \prod_a f_a(X_a)$$

### Elements

| Element | Representation | Description |
|---------|---------------|-------------|
| **Variable** | Cercle | Une quantite aleatoire |
| **Facteur** | Carre | Une fonction reliant des variables |
| **Arete** | Ligne | Connexion variable-facteur |

### Exemple visuel

```
     [Prior A]     [Prior B]
         |             |
        (A)           (B)
         |             |
         +-----[AND]---+
                |
               (C)
```

Ce graphe represente : $P(A, B, C) = P(A) \cdot P(B) \cdot P(C|A,B)$

## 3. Murder Mystery - Scenario

### Contexte (MBML Book, Chapter 1)

Un meurtre a ete commis dans un manoir victorien. Deux suspects :

| Suspect | Description | Prior de culpabilite |
|---------|-------------|---------------------|
| **Major Auburn** | Homme militaire, cheveux auburn | 70% |
| **Miss Grey** | Jeune femme, cheveux gris | 30% |

### Indices

1. **Arme du crime** : Revolver ou Dague
   - Auburn utiliserait un revolver (90%) ou une dague (10%)
   - Grey utiliserait un revolver (20%) ou une dague (80%)

2. **Cheveu trouve** : Auburn ou Gris
   - Si Auburn coupable : cheveu auburn (80%) ou gris (20%)
   - Si Grey coupable : cheveu auburn (10%) ou gris (90%)

### Question

L'enquete revele : **arme = revolver**, **cheveu = gris**. Qui est le meurtrier ?

## 4. Modele avec Prior Seulement

In [ ]:
// Modele 1 : Prior seulement
// meurtrier = true -> Auburn, false -> Grey

Variable<bool> meurtrier = Variable.Bernoulli(0.7);  // 70% Auburn

InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Bernoulli posteriorMeurtrier = moteur.Infer<Bernoulli>(meurtrier);

Console.WriteLine("=== Modele 1 : Prior seulement ===");
Console.WriteLine($"P(Auburn coupable) = {posteriorMeurtrier.GetProbTrue():F2}");
Console.WriteLine($"P(Grey coupable) = {1 - posteriorMeurtrier.GetProbTrue():F2}");

Compiling model...

done.


=== Modele 1 : Prior seulement ===


P(Auburn coupable) = 0,70


P(Grey coupable) = 0,30


### Interpretation du prior

Sans aucun indice, le prior reflète simplement notre croyance initiale : Auburn est considéré a priori plus susceptible d'être coupable (70%) que Grey (30%). Ce prior pourrait venir d'informations préalables (antécédents, mobile, etc.)

> **Point clé** : Le prior est **subjectif** mais l'inférence bayésienne garantit une mise à jour **cohérente** avec les observations. Un prior différent donnerait un postérieur différent, mais la direction des mises à jour resterait la même.

## 5. Ajout de l'Evidence sur l'Arme

In [ ]:
// Modele 2 : Prior + Arme
// arme = true -> Revolver, false -> Dague

Variable<bool> meurtrier2 = Variable.Bernoulli(0.7);
Variable<bool> arme = Variable.New<bool>();

// Si Auburn (meurtrier2 = true) : 90% revolver, 10% dague
using (Variable.If(meurtrier2))
{
    arme.SetTo(Variable.Bernoulli(0.9));  // 90% revolver
}

// Si Grey (meurtrier2 = false) : 20% revolver, 80% dague
using (Variable.IfNot(meurtrier2))
{
    arme.SetTo(Variable.Bernoulli(0.2));  // 20% revolver
}

// Observation : l'arme est un revolver
arme.ObservedValue = true;

InferenceEngine moteur2 = new InferenceEngine();
moteur2.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Bernoulli posteriorMeurtrier2 = moteur2.Infer<Bernoulli>(meurtrier2);

Console.WriteLine("=== Modele 2 : Prior + Arme (revolver) ===");
Console.WriteLine($"P(Auburn coupable | arme=revolver) = {posteriorMeurtrier2.GetProbTrue():F3}");
Console.WriteLine($"P(Grey coupable | arme=revolver) = {1 - posteriorMeurtrier2.GetProbTrue():F3}");

Compiling model...

done.


=== Modele 2 : Prior + Arme (revolver) ===


P(Auburn coupable | arme=revolver) = 0,913


P(Grey coupable | arme=revolver) = 0,087


### Analyse de l'evidence "arme"

**Résultat** : P(Auburn | revolver) = 0.913

Le revolver est un **indice très discriminant** car :
- Auburn utiliserait un revolver dans **90%** des cas
- Grey utiliserait un revolver dans seulement **20%** des cas

**Calcul du rapport de vraisemblance (likelihood ratio)** :
$$\text{LR} = \frac{P(\text{revolver}|\text{Auburn})}{P(\text{revolver}|\text{Grey})} = \frac{0.9}{0.2} = 4.5$$

Ce rapport de 4.5 indique que l'observation du revolver est **4.5 fois plus probable** si Auburn est coupable. Combiné avec le prior (70/30 = 2.33), les odds finaux sont 2.33 × 4.5 ≈ 10.5, soit ~91% pour Auburn.

> **Formule de Bayes en odds** : Posterior odds = Prior odds × Likelihood ratio

## 6. Ajout de l'Evidence sur les Cheveux

In [ ]:
// Modele 3 : Prior + Arme + Cheveu
// cheveu = true -> Auburn, false -> Gris

Variable<bool> meurtrier3 = Variable.Bernoulli(0.7);
Variable<bool> arme3 = Variable.New<bool>();
Variable<bool> cheveu = Variable.New<bool>();

// Probabilites conditionnelles pour l'arme
using (Variable.If(meurtrier3))
{
    arme3.SetTo(Variable.Bernoulli(0.9));
}
using (Variable.IfNot(meurtrier3))
{
    arme3.SetTo(Variable.Bernoulli(0.2));
}

// Probabilites conditionnelles pour le cheveu
// Si Auburn : 80% cheveu auburn, 20% cheveu gris
using (Variable.If(meurtrier3))
{
    cheveu.SetTo(Variable.Bernoulli(0.8));  // 80% auburn
}
// Si Grey : 10% cheveu auburn, 90% cheveu gris
using (Variable.IfNot(meurtrier3))
{
    cheveu.SetTo(Variable.Bernoulli(0.1));  // 10% auburn
}

// Observations
arme3.ObservedValue = true;   // Revolver
cheveu.ObservedValue = false; // Cheveu gris

InferenceEngine moteur3 = new InferenceEngine();
moteur3.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Bernoulli posteriorMeurtrier3 = moteur3.Infer<Bernoulli>(meurtrier3);

Console.WriteLine("=== Modele 3 : Prior + Arme (revolver) + Cheveu (gris) ===");
Console.WriteLine($"P(Auburn coupable | arme=revolver, cheveu=gris) = {posteriorMeurtrier3.GetProbTrue():F3}");
Console.WriteLine($"P(Grey coupable | arme=revolver, cheveu=gris) = {1 - posteriorMeurtrier3.GetProbTrue():F3}");

Compiling model...

done.


=== Modele 3 : Prior + Arme (revolver) + Cheveu (gris) ===


P(Auburn coupable | arme=revolver, cheveu=gris) = 0,700


P(Grey coupable | arme=revolver, cheveu=gris) = 0,300


### Analyse des evidences contradictoires

**Résultat** : P(Auburn | revolver, cheveu gris) = 0.700 — retour au prior !

Les deux indices s'**annulent exactement** dans ce modèle calibré :

| Indice | Likelihood Ratio | Direction |
|--------|------------------|-----------|
| Revolver | 0.9/0.2 = **4.5** | → Auburn |
| Cheveu gris | 0.2/0.9 = **0.22** | → Grey |

**Produit des LR** : 4.5 × 0.22 ≈ 1.0

Quand le produit des rapports de vraisemblance vaut 1, les indices sont **parfaitement compensatoires** et on revient au prior.

> **Leçon** : L'inférence bayésienne combine **tous** les indices de manière cohérente, même quand ils pointent dans des directions opposées. C'est une propriété fondamentale : aucun indice n'est ignoré, mais leur impact relatif dépend de leur force discriminante.

## 7. Le Paradoxe de Monty Hall

### Regles du jeu

1. Un jeu televisé presente **3 portes** : derriere l'une se trouve une voiture, derriere les deux autres des chevres
2. Le joueur **choisit une porte**
3. L'animateur (Monty), qui sait ou est la voiture, **ouvre une autre porte** montrant une chevre
4. Monty propose au joueur de **changer de porte**

### Question

Le joueur devrait-il changer de porte ?

### Intuition (souvent fausse)

Beaucoup pensent que la probabilite est 50/50 apres l'ouverture d'une porte. C'est faux !

In [ ]:
// Modelisation du probleme de Monty Hall

// Position de la voiture (0, 1, ou 2)
Variable<int> voiture = Variable.DiscreteUniform(3);

// Choix du joueur (0, 1, ou 2)
Variable<int> choixJoueur = Variable.DiscreteUniform(3);

// Porte ouverte par Monty
// Monty choisit une porte qui n'est ni la voiture, ni le choix du joueur
Variable<int> porteOuverte = Variable.New<int>();

// Logique de Monty : ouvrir une porte avec une chevre, differente du choix
// Cas 1 : voiture = 0
using (Variable.Case(voiture, 0))
{
    using (Variable.Case(choixJoueur, 0))
    {
        // Joueur a choisi 0 (voiture), Monty peut ouvrir 1 ou 2
        porteOuverte.SetTo(Variable.DiscreteUniform(2) + 1);  // 1 ou 2 avec prob egale
    }
    using (Variable.Case(choixJoueur, 1))
    {
        // Joueur a choisi 1, Monty doit ouvrir 2 (pas 0=voiture, pas 1=choix)
        porteOuverte.SetTo(Variable.Constant(2));
    }
    using (Variable.Case(choixJoueur, 2))
    {
        // Joueur a choisi 2, Monty doit ouvrir 1
        porteOuverte.SetTo(Variable.Constant(1));
    }
}

// Cas 2 : voiture = 1
using (Variable.Case(voiture, 1))
{
    using (Variable.Case(choixJoueur, 0))
    {
        porteOuverte.SetTo(Variable.Constant(2));
    }
    using (Variable.Case(choixJoueur, 1))
    {
        // Joueur a choisi la voiture, Monty peut ouvrir 0 ou 2
        porteOuverte.SetTo(Variable.Discrete(new double[] { 0.5, 0.0, 0.5 }));
    }
    using (Variable.Case(choixJoueur, 2))
    {
        porteOuverte.SetTo(Variable.Constant(0));
    }
}

// Cas 3 : voiture = 2
using (Variable.Case(voiture, 2))
{
    using (Variable.Case(choixJoueur, 0))
    {
        porteOuverte.SetTo(Variable.Constant(1));
    }
    using (Variable.Case(choixJoueur, 1))
    {
        porteOuverte.SetTo(Variable.Constant(0));
    }
    using (Variable.Case(choixJoueur, 2))
    {
        // Joueur a choisi la voiture, Monty peut ouvrir 0 ou 1
        porteOuverte.SetTo(Variable.Discrete(new double[] { 0.5, 0.5, 0.0 }));
    }
}

Console.WriteLine("Modele Monty Hall defini.");

Modele Monty Hall defini.


In [ ]:
// Scenario : Le joueur choisit la porte 0, Monty ouvre la porte 2
choixJoueur.ObservedValue = 0;
porteOuverte.ObservedValue = 2;

InferenceEngine moteurMonty = new InferenceEngine();
moteurMonty.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Discrete posteriorVoiture = moteurMonty.Infer<Discrete>(voiture);

Console.WriteLine("=== Paradoxe de Monty Hall ===");
Console.WriteLine($"Choix du joueur : porte 0");
Console.WriteLine($"Porte ouverte par Monty : porte 2 (chevre)\n");

Console.WriteLine($"P(voiture derriere porte 0) = {posteriorVoiture.GetProbs()[0]:F3}");
Console.WriteLine($"P(voiture derriere porte 1) = {posteriorVoiture.GetProbs()[1]:F3}");
Console.WriteLine($"P(voiture derriere porte 2) = {posteriorVoiture.GetProbs()[2]:F3}");

Console.WriteLine($"\n=> Le joueur devrait CHANGER pour la porte 1 !");
Console.WriteLine($"   En gardant : {posteriorVoiture.GetProbs()[0]*100:F0}% de chance");
Console.WriteLine($"   En changeant : {posteriorVoiture.GetProbs()[1]*100:F0}% de chance");

Compiling model...

done.


=== Paradoxe de Monty Hall ===


Choix du joueur : porte 0


Porte ouverte par Monty : porte 2 (chevre)



...

P(voiture derriere porte 1) = 0,667


P(voiture derriere porte 2) = 0,000



=> Le joueur devrait CHANGER pour la porte 1 !


   En gardant : 33% de chance


   En changeant : 67% de chance


### Analyse du résultat Monty Hall

**Sortie** : P(voiture porte 0) = 0.333, P(voiture porte 1) = **0.667**, P(voiture porte 2) = 0

L'information révélée par Monty n'est **pas symétrique** :
- Si le joueur a choisi la voiture (1/3), Monty pouvait ouvrir n'importe quelle autre porte
- Si le joueur a choisi une chèvre (2/3), Monty était **contraint** d'ouvrir l'unique autre porte avec une chèvre

C'est cette **asymétrie d'information** qui crée le paradoxe. L'action de Monty révèle indirectement où est la voiture.

> **Intuition correcte** : En changeant, vous captez les 2/3 de chance initiale que la voiture soit derrière l'une des portes non choisies. Avant l'ouverture, cette probabilité était répartie sur 2 portes. Après, elle se concentre sur une seule.

### Explication du Paradoxe

| Scenario initial | Probabilite | Resultat si change |
|------------------|-------------|--------------------|
| Joueur choisit la voiture | 1/3 | Perd (change vers chevre) |
| Joueur choisit une chevre | 2/3 | **Gagne** (change vers voiture) |

**Conclusion** : En changeant, le joueur gagne avec probabilite **2/3** au lieu de 1/3 !

## 8. Theoreme de Bayes Illustre

### Formule

$$P(H|E) = \frac{P(E|H) \cdot P(H)}{P(E)}$$

Ou :
- $P(H)$ : Prior (croyance initiale en l'hypothese)
- $P(E|H)$ : Vraisemblance (probabilite de l'evidence si H est vrai)
- $P(H|E)$ : Posterieur (croyance mise a jour)
- $P(E)$ : Evidence marginale (normalisation)

### Application au Murder Mystery

$$P(\text{Auburn}|\text{Revolver}) = \frac{P(\text{Revolver}|\text{Auburn}) \cdot P(\text{Auburn})}{P(\text{Revolver})}$$

In [ ]:
// Verification manuelle du theoreme de Bayes

double pAuburn = 0.7;  // Prior
double pGrey = 0.3;

double pRevolverSiAuburn = 0.9;  // Vraisemblance
double pRevolverSiGrey = 0.2;

// P(Revolver) = P(R|A)*P(A) + P(R|G)*P(G)
double pRevolver = pRevolverSiAuburn * pAuburn + pRevolverSiGrey * pGrey;

// P(Auburn|Revolver) par Bayes
double pAuburnSiRevolver = (pRevolverSiAuburn * pAuburn) / pRevolver;

Console.WriteLine("=== Verification manuelle de Bayes ===");
Console.WriteLine($"P(Revolver) = {pRevolver:F3}");
Console.WriteLine($"P(Auburn|Revolver) = {pAuburnSiRevolver:F3}");
Console.WriteLine($"\nCeci correspond au resultat d'Infer.NET !");

=== Verification manuelle de Bayes ===


P(Revolver) = 0,690


P(Auburn|Revolver) = 0,913



Ceci correspond au resultat d'Infer.NET !


## 9. Pattern Variable.If / Variable.IfNot

### Syntaxe

```csharp
Variable<bool> condition = Variable.Bernoulli(0.5);
Variable<double> resultat = Variable.New<double>();

using (Variable.If(condition))
{
    resultat.SetTo(Variable.GaussianFromMeanAndPrecision(10, 1));
}

using (Variable.IfNot(condition))
{
    resultat.SetTo(Variable.GaussianFromMeanAndPrecision(20, 1));
}
```

### Applications

| Pattern | Usage |
|---------|-------|
| `Variable.If` / `Variable.IfNot` | Branchement binaire |
| `Variable.Case` | Branchement discret multi-valeur |
| `Variable.Switch` | Selection dans un tableau |

## 10. Exercice : Ajouter un Temoin

### Enonce

Un temoin declare avoir vu le meurtrier s'enfuir. Sa fiabilite :
- S'il a bien vu Auburn : 70% de chance de dire "homme", 30% de dire "femme"
- S'il a bien vu Grey : 20% de chance de dire "homme", 80% de dire "femme"

Le temoin dit avoir vu un **homme**.

### Question

Avec les indices (arme=revolver, cheveu=gris, temoin=homme), quelle est la probabilite de culpabilite ?

### Indice

Ajoutez une variable `temoin` avec les probabilites conditionnelles appropriees.

In [ ]:
// EXERCICE : Modele complet avec temoin

Variable<bool> meurtrierEx = Variable.Bernoulli(0.7);
Variable<bool> armeEx = Variable.New<bool>();
Variable<bool> cheveuEx = Variable.New<bool>();
Variable<bool> temoin = Variable.New<bool>();  // true = homme, false = femme

// Arme
using (Variable.If(meurtrierEx))
{
    armeEx.SetTo(Variable.Bernoulli(0.9));
}
using (Variable.IfNot(meurtrierEx))
{
    armeEx.SetTo(Variable.Bernoulli(0.2));
}

// Cheveu
using (Variable.If(meurtrierEx))
{
    cheveuEx.SetTo(Variable.Bernoulli(0.8));
}
using (Variable.IfNot(meurtrierEx))
{
    cheveuEx.SetTo(Variable.Bernoulli(0.1));
}

// Temoin : P(homme|Auburn) = 0.7, P(homme|Grey) = 0.2
using (Variable.If(meurtrierEx))
{
    temoin.SetTo(Variable.Bernoulli(0.7));  // 70% dit homme si Auburn
}
using (Variable.IfNot(meurtrierEx))
{
    temoin.SetTo(Variable.Bernoulli(0.2));  // 20% dit homme si Grey
}

// Observations
armeEx.ObservedValue = true;   // Revolver
cheveuEx.ObservedValue = false; // Cheveu gris
temoin.ObservedValue = true;   // Temoin dit "homme"

InferenceEngine moteurEx = new InferenceEngine();
moteurEx.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Bernoulli posteriorEx = moteurEx.Infer<Bernoulli>(meurtrierEx);

Console.WriteLine("=== Modele complet avec temoin ===");
Console.WriteLine($"Indices : arme=revolver, cheveu=gris, temoin=homme\n");
Console.WriteLine($"P(Auburn coupable) = {posteriorEx.GetProbTrue():F3}");
Console.WriteLine($"P(Grey coupable) = {1 - posteriorEx.GetProbTrue():F3}");

Compiling model...

done.


=== Modele complet avec temoin ===


Indices : arme=revolver, cheveu=gris, temoin=homme



P(Auburn coupable) = 0,891


P(Grey coupable) = 0,109


### Analyse de l'exercice avec témoin

**Résultat** : P(Auburn | revolver, cheveu gris, témoin homme) = **0.891**

Le témoignage "homme" a un **fort impact** car son likelihood ratio est significatif :
$$\text{LR}_{\text{témoin}} = \frac{P(\text{homme}|\text{Auburn})}{P(\text{homme}|\text{Grey})} = \frac{0.7}{0.2} = 3.5$$

**Évolution de la probabilité** :

| Étape | P(Auburn) | Indices |
|-------|-----------|---------|
| Prior | 0.70 | Aucun |
| +Revolver | 0.91 | Arme seule |
| +Cheveu gris | 0.70 | Arme + Cheveu (annulation) |
| +Témoin homme | **0.89** | Tous les indices |

Le témoin rompt l'équilibre créé par les deux premiers indices contradictoires. Avec trois indices, deux pointent vers Auburn (revolver, témoin) contre un seul vers Grey (cheveu), d'où une probabilité finale fortement en faveur d'Auburn.

## 11. Resume

| Concept | Description |
|---------|-------------|
| **Graphe de facteurs** | Representation graphique d'une distribution jointe |
| **Variable.If/IfNot** | Conditionnement binaire |
| **Variable.Case** | Conditionnement discret multi-valeur |
| **Theoreme de Bayes** | Mise a jour des croyances avec l'evidence |
| **Murder Mystery** | Exemple classique d'inference discrete |
| **Monty Hall** | Paradoxe illustrant l'importance du conditionnement |

---

## Prochaine etape

Dans [Infer-4-Bayesian-Networks](Infer-4-Bayesian-Networks.ipynb), nous approfondirons :

- Les reseaux bayesiens classiques (Wet Grass / Sprinkler / Rain)
- Les tables de probabilites conditionnelles (CPT)
- L'inference causale vs observationnelle
- La D-separation et l'independance conditionnelle